In [1]:
from sklearn.neural_network import MLPClassifier
import sklearn.metrics as metrics
import pandas as pd
import json
import os
import codecs
from unidecode import unidecode
import pickle
import string
import decimal
import sys
import pickle
import pyodbc
import sys
import sklearn
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.externals import joblib
from pandas.io.sql import read_sql
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from nltk.stem.snowball import PortugueseStemmer
import re
import math
import numpy as np
import os.path
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn import preprocessing
import codecs
from sklearn.svm import SVR
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import cross_val_score
from sklearn import feature_selection
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
import math
from catboost import CatBoostRegressor

In [2]:
def json_to_dataframe(path):
    with codecs.open(path, 'r', encoding='utf-8') as reader:
        d = json.load(reader)
    return pd.DataFrame.from_dict(d)

In [3]:
def l2normalize(xdf):
    return pd.DataFrame(preprocessing.normalize(xdf, norm='l2', axis=1, copy=True, return_norm=False),
             columns=xdf.columns, index=xdf.index)

In [4]:
def print_result(gold_values, sys_values, name):
    """Docstring."""
    pearson = pearsonr(gold_values, sys_values)[0]
    spearman = spearmanr(gold_values, sys_values)[0]
    absolute_diff = gold_values - sys_values
    mse = (absolute_diff ** 2).mean()

    print()
    print('Similarity evaluation\t' + name)
    print('Pearson\t\tSpearman\t\tMean Squared Error')
    print('-------\t\t-------\t\t------------------')
    print('{:7.2f}\t\t{:7.2f}\t\t{:18.2f}'.format(pearson,spearman,mse))

In [5]:
def dump_json(outputpath, x):
    t = dict()
    t['y_predicted'] = x.tolist()
    with codecs.open(outputpath, 'w', encoding='utf-8') as writer:
        json.dump(t, writer)

In [6]:
homepath = 'C:/Users/barcelosallan/{0}/{1}'
default_settings = 'Dropbox/unisinos-pipca-aleksejs-allan-sandro/Data/Processed/GloVe/'
synonym_settings = 'Dropbox/unisinos-pipca-aleksejs-allan-sandro/Data/Processed/GloVe/linguistic_relations/Synonyms/'
synonym_hyperonym_settings = 'Dropbox/unisinos-pipca-aleksejs-allan-sandro/Data/Processed/GloVe/linguistic_relations/Synonyms_and_Hyperonyms/'
predictions_settings = 'Dropbox/unisinos-pipca-aleksejs-allan-sandro/Predictions/Glove/NILC_300million_1000window'

extension_processed_file = 'svm-clear-setEP{0}_setEP{1}.json'
extension_synonym_file = 'setP{0}-synonym_processed.json'
extension_hyponym = 'setE{0}-synonym-hyperonym_processed.json'
extension_processed_synonym_file = 'svm-synonym-setEP{0}_setEP{1}.json'
extension_processed_hyperonym_file = 'svm-synonym_hyperonym-setEP{0}_setEP{1}.json'
extension_default_file = 'setP{0}_processed.json'
extension_final_file = 'A3_{0}.json'


In [7]:
setEPX = json_to_dataframe(homepath.format(synonym_settings, extension_synonym_file.format(1)))

In [8]:
setEPY = json_to_dataframe(homepath.format(synonym_settings, extension_synonym_file.format(3)))

In [9]:
drop_columns = ['index', 'antonyms']

setEPX.sort_values('index', inplace=True, ascending=True)
setEPY.sort_values('index', inplace=True, ascending=True)

for x in drop_columns:
    if x in setEPX.columns:
        setEPX.drop(x, axis=1, inplace=True)
        setEPY.drop(x, axis=1, inplace=True)

In [10]:
x_train = setEPX.drop('target', 1)
y_train = [float(x) for x in setEPX['target'].tolist()]
x_test = setEPY.drop('target', 1)
y_test = [float(x) for x in setEPY['target'].tolist()]

In [11]:
#x_train = l2normalize(x_train)
#x_test = l2normalize(x_test)

In [12]:
x_train = x_train[['embeddings_diff', 'tfidf', 'ngram_proportion', 'word_overlap']]
x_test = x_test[['embeddings_diff', 'tfidf', 'ngram_proportion', 'word_overlap']]

In [13]:
model_ml = SVR(kernel='poly', gamma=0.5, epsilon=0.1, C=1)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'SVR RBF')


Similarity evaluation	SVR RBF
Pearson		Spearman		Mean Squared Error
-------		-------		------------------
   0.66		   0.64		              0.43


In [14]:
model_ml = SVR(kernel='rbf')
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'SVR RBF')


Similarity evaluation	SVR RBF
Pearson		Spearman		Mean Squared Error
-------		-------		------------------
   0.65		   0.64		              0.44


In [15]:
model_ml = SVR(kernel='linear', C=1)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'SVR RBF')


Similarity evaluation	SVR RBF
Pearson		Spearman		Mean Squared Error
-------		-------		------------------
   0.65		   0.64		              0.44


In [16]:
model_ml = RandomForestRegressor(n_estimators=500,n_jobs=-1,bootstrap=True)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'RandomForest')


Similarity evaluation	RandomForest
Pearson		Spearman		Mean Squared Error
-------		-------		------------------
   0.64		   0.62		              0.45


In [17]:
model_ml = linear_model.LinearRegression(n_jobs=-1)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'LinearRegression')


Similarity evaluation	LinearRegression
Pearson		Spearman		Mean Squared Error
-------		-------		------------------
   0.66		   0.65		              0.43


In [18]:
model_ml = linear_model.Ridge()
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'RidgeRegression')


Similarity evaluation	RidgeRegression
Pearson		Spearman		Mean Squared Error
-------		-------		------------------
   0.65		   0.64		              0.44


In [19]:
model_ml = linear_model.BayesianRidge(n_iter=1000,normalize=False)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'BayesianRidgeRegression')


Similarity evaluation	BayesianRidgeRegression
Pearson		Spearman		Mean Squared Error
-------		-------		------------------
   0.66		   0.64		              0.43


In [20]:
model_ml = linear_model.ElasticNet(alpha=0.001, l1_ratio=0.5, fit_intercept=True, normalize=False, precompute=False, max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'ElasticNet')


Similarity evaluation	ElasticNet
Pearson		Spearman		Mean Squared Error
-------		-------		------------------
   0.65		   0.64		              0.44


In [21]:
model_ml = CatBoostRegressor(learning_rate=0.012, depth=1, iterations=2500, verbose=False)
y_predicted = model_ml.fit(x_train, y_train).predict(x_test)
print_result(y_test, y_predicted, 'CatBoostRegressor')


Similarity evaluation	CatBoostRegressor
Pearson		Spearman		Mean Squared Error
-------		-------		------------------
   0.66		   0.64		              0.43
